#### <h1> <center> ENSF 592 Programming Fundamentals for Data Engineers </center></h1>
<h2> <center> Assignment 7: Beautifulsoup and Pandas (5 points)</center></h2>
<center>
<div class="alert alert-block alert-info">
Due: Wed Nov 20 12am (midnight, Tue to Wed). To be submitted on D2L.
</div></center>




Edit this file and write your solutions to the problems in sections specified with `# Your solution goes here`. Test your code and when you are done, download this notebook as an `.ipynb` file and submit it to D2L. To get this file, in Jupyter notebook you can go to File -> Download as -> Notebook(.ipynb)

# Crawl and Scrape 

Schulich wants to have an integrated dataset of all engineering professors in one place. So as a data engineer, you're asked to gather some information of engineering professors by crawling different faculty website. Then, scrape their information and load them to a pandas dataframe and eventually save it as a csv file. 




In the first step, you need to get the html text of the website using `requests` library, and then you must use `Beautifulsoup4` library and `lxml` parser to pars the html and extract the needed information.     

## Section A.(2.5 pts)

In the following, all engineering faculties' websites are listed in the URLs. First, for each faculty, get the html text of the webpage and scrape the information of all its professors to put them in a dataframe as presented below: 




### You're DataFrame will have the following columns: 

|firsname|lastname|title|faculty|email|homepage|
|--------|--------|-----|--------|-----|--------|
|        |        |      |     |      |        |

<div class="alert alert-block alert-info">
<b>Tip:</b> Use `Inspect Element` of Chrome to see the mapping html tags to objects in a webpage<br>
    <i>Tutorial Link</i>: https://www.youtube.com/watch?v=1l4xz1QQhew&t=3s
</div>

In [68]:
from backports import os
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings(action="once")

In [74]:
all_urls=["https://schulich.ucalgary.ca/departments/electrical-and-computer-engineering/faculty",
"https://schulich.ucalgary.ca/departments/chemical-and-petroleum-engineering/faculty",
"https://schulich.ucalgary.ca/departments/civil-engineering/faculty",
"https://schulich.ucalgary.ca/departments/mechanical-and-manufacturing-engineering/faculty",
"https://schulich.ucalgary.ca/geomatics/faculty-members"]

df_professors=pd.DataFrame(columns = ["firstname", "lastname", "title", "faculty", "email","homepage"])

# Your solution goes here
for i in range(5):
    html=requests.get(all_urls[i], verify=False).text
    soup = BeautifulSoup(html, 'lxml')
    
    faculty = all_urls[i].split("/")[-2]
    print(faculty)
    # all profs are children of this special div, wrapped in a p
    for row in soup.find('div',class_="col-sm-12 two-col").find_all('p'):
        print(row)
        text = row.get_text()
        text = text.replace("\xa0"," ")
        
        try:
            the_name = re.findall("Dr\.\s([\w\s\(\)\.-]*),",text)[0].split()
            firstname = the_name[0]
            lastname = " ".join(the_name[1:])
            email=row.select('a[href^=mailto]')[0].get_text().strip()
            title = re.findall(f'{lastname}, ([\w\s,(),]*){email}',text)
            if len(title) == 1:
                title = title[0]
            print(rf"{lastname}")
            print(rf"{email}")
            print(title)
            if email == 'spjazaye@ucalgary.ca':
                title = 'Senior Instructor'
            if email == 'dean@ucalgary.ca':
                title = 'Professor, Dean'
            
            homepage=np.nan
            # firstname = row.find(property="profile:first_name").content
            
        
            # profile websites have a href that contains /contacts
            link=row.find(href=re.compile("/contacts"))
            if link is not None:
        #         print(link['href'])
                homepage="https://schulich.ucalgary.ca"+link['href']  
        
            # add a row
            df_professors=df_professors.append({'firstname': firstname,'lastname': lastname,'title': title,'faculty': faculty,'email': email,'homepage':homepage}, ignore_index=True)
        except:
            print([text])


df_professors

electrical-and-computer-engineering
<p><span><span><span>Dr. Norm Bartley, Senior Instructor<br/><a href="mailto:nbartley@ucalgary.ca">nbartley@ucalgary.ca</a><br/><a href="/contacts/norm-bartley">View profile &gt;</a></span></span></span></p>
Bartley
nbartley@ucalgary.ca
Senior Instructor
<p><span><span><span>Dr. Laleh Behjat, Professor<br/><a href="mailto:laleh@ucalgary.ca"><span><span>laleh@ucalgary.ca</span></span></a><br/><a href="/contacts/laleh-behjat">View profile &gt;</a></span></span></span></p>
Behjat
laleh@ucalgary.ca
Professor
<p><span><span><span>Dr. Leo Belostotski, Professor<br/><a href="mailto:lbelosto@ucalgary.ca">lbelosto@ucalgary.ca</a><br/><a href="/contacts/leo-belostotski">View profile &gt;</a></span></span></span></p>
Belostotski
lbelosto@ucalgary.ca
Professor
<p><span><span><span>Dr. </span></span></span>Laura Curiel<span><span><span>, Assistant Professor</span></span></span><br/><a href="mailto:laura.curiel@ucalgary.ca">laura.curiel@ucalgary.ca</a><br/><span><

,firstname,lastname,title,faculty,email,homepage
0,Norm,Bartley,Senior Instructor,electrical-and-computer-engineering,nbartley@ucalgary.ca,https://schulich.ucalgary.ca/contacts/norm-bar...
1,Laleh,Behjat,Professor,electrical-and-computer-engineering,laleh@ucalgary.ca,https://schulich.ucalgary.ca/contacts/laleh-be...
2,Leo,Belostotski,Professor,electrical-and-computer-engineering,lbelosto@ucalgary.ca,https://schulich.ucalgary.ca/contacts/leo-belo...
3,Laura,Curiel,Assistant Professor,electrical-and-computer-engineering,laura.curiel@ucalgary.ca,https://schulich.ucalgary.ca/contacts/laura-cu...
4,Colin,Dalton,Assistant Professor,electrical-and-computer-engineering,cdalton@ucalgary.ca,https://schulich.ucalgary.ca/contacts/colin-da...
...,...,...,...,...,...,...
169,Michael,Sideris,"Professor, Associate Head (Graduate Studies)",geomatics,sideris@ucalgary.ca,https://schulich.ucalgary.ca/contacts/michael-...
170,Susan,Skone,"Professor, Associate VP (Research)",geomatics,shskone@ucalgary.ca,NaN
171,Emmanuel,Stefanakis,"Professor, Department Head",geomatics,emmanuel.stefanakis@ucalgary.ca,https://schulich.ucalgary.ca/contacts/emmanuel...
172,Xin,Wang,Professor,geomatics,xcwang@ucalgary.ca,https://schulich.ucalgary.ca/contacts/xin-wang


## Section B (2.5 pts)

### part I (2 pts)
In this part, iterate on professors dataframe and request to get their homepage html, and find the phone number and office (building and room) of each professor and add it to your previous dataframe as a new column.  Finally, save the dataframe as a csv file. 


<div class="alert alert-block alert-danger">
<b>Note:</b> <br>
Put `nan` for professors who do not have phone and office numbers on their webpage
</div>



In [105]:
# Your solution goes here
with requests.Session() as s:
    page = df_professors.loc[:, "homepage"]
    for i in range(df_professors.shape[0]):
        try:
            print(page[i])
            html=s.get(page[i], verify=False).text
            soup = BeautifulSoup(html, 'lxml')
            text = str(soup.prettify())
            phone = re.findall("403?[\s.-]?\d{3}[\s.-]?\d{4}",text)
            if phone:
                df_professors.ix[i, 'Phone Number'] = phone[0] 
            # phone = re.findall("403?[\s.-]?\d{3}[\s.-]?\d{4}",text)
            building = re.findall("https:\/\/www\.ucalgary\.ca\/map\/contactlist\/\?Building=(.*)",text)
            if building:
                df_professors.ix[i, 'Office'] = building[0].replace("&amp;Room=", " ").replace("\">", "").replace("%20", "")
        except:
            print(page[i])
            

https://schulich.ucalgary.ca/contacts/norm-bartley
https://schulich.ucalgary.ca/contacts/laleh-behjat
https://schulich.ucalgary.ca/contacts/leo-belostotski
https://schulich.ucalgary.ca/contacts/laura-curiel
https://schulich.ucalgary.ca/contacts/colin-dalton
https://schulich.ucalgary.ca/contacts/vassil-dimitrov
https://schulich.ucalgary.ca/contacts/abraham-fapojuwo
https://schulich.ucalgary.ca/contacts/behrouz-far
https://schulich.ucalgary.ca/contacts/elise-fear
https://schulich.ucalgary.ca/contacts/fadhel-ghannouchi
https://schulich.ucalgary.ca/contacts/anis-haque
https://schulich.ucalgary.ca/contacts/mohamed-helaoui
https://schulich.ucalgary.ca/contacts/hadi-hemmati
https://schulich.ucalgary.ca/contacts/yaoping-hu
https://schulich.ucalgary.ca/contacts/seyed-pouyan-yani-jazayeri
https://schulich.ucalgary.ca/contacts/andy-knight
https://schulich.ucalgary.ca/contacts/diwakar-krishnamurthy
https://schulich.ucalgary.ca/contacts/henry-leung
https://schulich.ucalgary.ca/contacts/chris-macnab

/home/bryce/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


In [106]:
import os
df_professors.to_csv(r'/home/bryce/schulich_profs.csv')      
    

    

### part II (0.5 pt)

In this part, you need to generate the following reports:
  
  
* Number of Full professors
* Number of Associate professors
* Number of Assistant professors
* Number of Instructor professors (Instructor, Senior Instructor, Teaching Professors)

In [102]:
# Your solution goes here
print(df_professors.shape[0])
professors = df_professors.title.str.contains("Professor").sum()
assistant_professors = df_professors.title.str.contains("Assistant Professor").sum()
associate_professors = df_professors.title.str.contains("Associate Professor").sum()
instructor_professors = df_professors.title.str.contains("Teaching|Instructor").sum()
print("Professors: " + str( professors-assistant_professors-associate_professors-instructor_professors))
print("Assistant Professors: " + str(assistant_professors))
print("Associate Professors: " + str(associate_professors))
print("Instructor Professors: " + str(instructor_professors))



174
Professors: 76
Assistant Professors: 30
Associate Professors: 31
Instructor Professors: 15
228
